In [18]:
!pwd

/data/home/nikolay.chernavin


In [19]:
!hdfs dfs -ls /labs/laba01/ml-100k

Found 23 items
-rw-r--r--   3 hdfs hdfs       6750 2022-01-06 18:46 /labs/laba01/ml-100k/README
-rw-r--r--   3 hdfs hdfs        716 2022-01-06 18:46 /labs/laba01/ml-100k/allbut.pl
-rw-r--r--   3 hdfs hdfs        643 2022-01-06 18:46 /labs/laba01/ml-100k/mku.sh
-rw-r--r--   3 hdfs hdfs    1979173 2022-01-06 18:46 /labs/laba01/ml-100k/u.data
-rw-r--r--   3 hdfs hdfs        202 2022-01-06 18:46 /labs/laba01/ml-100k/u.genre
-rw-r--r--   3 hdfs hdfs         36 2022-01-06 18:46 /labs/laba01/ml-100k/u.info
-rw-r--r--   3 hdfs hdfs     236344 2022-01-06 18:46 /labs/laba01/ml-100k/u.item
-rw-r--r--   3 hdfs hdfs        193 2022-01-06 18:46 /labs/laba01/ml-100k/u.occupation
-rw-r--r--   3 hdfs hdfs      22628 2022-01-06 18:46 /labs/laba01/ml-100k/u.user
-rw-r--r--   3 hdfs hdfs    1586544 2022-01-06 18:46 /labs/laba01/ml-100k/u1.base
-rw-r--r--   3 hdfs hdfs     392629 2022-01-06 18:46 /labs/laba01/ml-100k/u1.test
-rw-r--r--   3 hdfs hdfs    1583948 2022-01-06 18:46 /labs/laba01/ml-100k/u2.base


In [29]:
import os
import sys

os.environ['PYSPARK PYTHON']='/opt/anaconda/envs/bd9/bin/python'
os.environ['PYSPARK_DRIVER_PYTHON']='/opt/anaconda/envs/bd9/bin/ipython'

os.environ['SPARK_HOME']='/usr/hdp/current/spark2-client'
os.environ['PYSPARK_SUBMIT_ARGS']='--num-executors 2 pyspark-shell'

spark_home=os.environ.get('SPARK_HOME',None)

if not spark_home:
    raise ValueError('SPARK_HOME environment variable is not set')
    
sys.path.insert(0,os.path.join(spark_home,'python'))
sys.path.insert(0,os.path.join(spark_home,'python/lib/py4j-0.10.7-src.zip'))                
                
exec(open(os.path.join(spark_home,'python/pyspark/shell.py')).read())

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 2.4.7
      /_/

Using Python version 3.6.5 (default, Apr 29 2018 16:14:56)
SparkSession available as 'spark'.


In [30]:
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import *
from pyspark import Row
import json
import operator

conf=SparkConf()

spark=(SparkSession
      .builder
      .config(conf=conf)
      .appName('test')
      .getOrCreate())

In [32]:
rdd=sc.textFile("/labs/laba01/ml-100k/u.data").map(lambda x: x.split("\t"))

In [77]:
rate_all=rdd.flatMap(lambda x: x[2].split()) #All films
rate_film=rdd.filter(lambda x: x[1]=='318').flatMap(lambda x: x[2].split()) #One film

In [117]:
rate_all_sort=[x[1] for x in sorted(rate_all.countByKey().items())] #All films sort
rate_film_sort=[x[1] for x in sorted(rate_film.countByKey().items())] #One film sort

In [132]:
spark.stop()

In [131]:
with open('lab01.json', 'w') as outfile:
    json.dump({"hist_film": rate_film_sort,"hist_all":rate_all_sort}, outfile)